# Data Collection

In this notebook, I'll scrape the data from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M and create a Pandas dataframe from it.

## Scraping data

I'll begin by scraping the table on the webpage.

I'll import `pandas` to create the dataframe, `urlopen` to make a request and `BeautifulSoup` to read through the HTML page.

In [1]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup

I'll load the link and fetch the table from the HTML. The first table on the page is the one I will be using.

In [2]:
html = urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
content = BeautifulSoup(html, 'html.parser')
table = content.find('table')
print(table.prettify())

<table class="wikitable sortable">
 <tbody>
  <tr>
   <th>
    Postcode
   </th>
   <th>
    Borough
   </th>
   <th>
    Neighbourhood
   </th>
  </tr>
  <tr>
   <td>
    M1A
   </td>
   <td>
    Not assigned
   </td>
   <td>
    Not assigned
   </td>
  </tr>
  <tr>
   <td>
    M2A
   </td>
   <td>
    Not assigned
   </td>
   <td>
    Not assigned
   </td>
  </tr>
  <tr>
   <td>
    M3A
   </td>
   <td>
    <a href="/wiki/North_York" title="North York">
     North York
    </a>
   </td>
   <td>
    <a href="/wiki/Parkwoods" title="Parkwoods">
     Parkwoods
    </a>
   </td>
  </tr>
  <tr>
   <td>
    M4A
   </td>
   <td>
    <a href="/wiki/North_York" title="North York">
     North York
    </a>
   </td>
   <td>
    <a href="/wiki/Victoria_Village" title="Victoria Village">
     Victoria Village
    </a>
   </td>
  </tr>
  <tr>
   <td>
    M5A
   </td>
   <td>
    <a href="/wiki/Downtown_Toronto" title="Downtown Toronto">
     Downtown Toronto
    </a>
   </td>
   <td>
    <a href="

Next, I'll iterate through all rows in the table except the first (which is the header) and save the contents to the variable `data`. I'll then use `data` to create the Pandas dataframe.

In [3]:
data = []
for tr in table.find_all('tr')[1:]:
    row_data = tr.find_all('td')
    data.append([cell.text for cell in row_data])
dataset = pd.DataFrame(data, columns = ['PostalCode', 'Borough', 'Neighborhood'])
dataset.head(5)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


## Cleaning the data

As I have created the Pandas dataframe, I'll clean the dataframe.

First, I'll remove all the unnecessary rows where `Borough` is `Not assigned`

In [4]:
dataset = dataset[dataset['Borough'] != 'Not assigned']
dataset.reset_index(drop = True, inplace = True)

Next, I'll assign `Neighbourhood` to same as `Borough` if it is `Not assigned`. However, the data in the column `Neighbourhood` has `\n` at the end of each name, so I'll remove it first and then do the assignment.

In [5]:
dataset['Neighborhood'] = dataset['Neighborhood'].str.split('\n', expand = True)[0]
dataset.loc[dataset['Neighborhood'] == 'Not assigned', 'Neighborhood'] = dataset.loc[dataset['Neighborhood'] == 'Not assigned', 'Borough']

Lastly, I'll combine the rows with the same `PostCode` and `Borough`.

In [6]:
dataset = dataset.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()

# Data saving

Finally, I'll save the data to a `.csv` file and display it's shape too.

In [7]:
dataset.to_csv('dataset.csv', index = False)

In [8]:
print("Dataset shape: {}".format(dataset.shape))

Dataset shape: (103, 3)
